In [1]:
from eventvision import read_dataset, read_annotation

import os
import cv2
import glob
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.patches import Rectangle
from sklearn.model_selection import train_test_split
import scipy.io

from BoundingBoxRegressionUtils import makeModel, parse_random_dataset, parse_partial_dataset, tf_iou

INPUT_SHAPE = keras.Input(shape=(128, 128, 3))

import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

Event-based vision module imported


In [2]:
def makeBB(filename):
    mat = scipy.io.loadmat(filename)
    data = np.array(mat["box_coord"][0])
    minX = data[2]
    maxX = data[3]
    minY = data[0]
    maxY = data[1]
    
    ax = plt.gca()
    return Rectangle((minX, minY), maxX - minX, maxY - minY, edgecolor='red', facecolor='none', label="ground truth")

def getData(filename):
    img = load_img(filename)
    
    data = img_to_array(img)
    arr = np.array(data)
    width = arr.shape[1]
    height = arr.shape[0]
    
    arr = cv2.resize(arr, (128,128)) / 256
    

    return width, height, arr

def getAnnotation(filename):
    mat = scipy.io.loadmat(filename)
    data = np.array(mat["box_coord"][0])
    minX = data[2]
    maxX = data[3]
    minY = data[0]
    maxY = data[1]
    
    return minX, minY, maxX, maxY

In [3]:
def testDataTime():
    data_results = np.empty(20)
    
    for i in range(20):
        data_start_time = time.process_time() 

        X_train, X_test, y_train, y_test = parse_partial_dataset(getData, getAnnotation, False)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

        data_time = time.process_time() - data_start_time
        
        data_results[i] = data_time
        
        print("Round ", i, " time ", data_time)
    
    return np.mean(data_results)

# print("data avg", testDataTime())

In [4]:
X_train, X_test, y_train, y_test = parse_partial_dataset(getData, getAnnotation, False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

def test(model, test_data):
    predictions = model.predict(test_data[0])
    
    bb_scores = tf_iou(predictions[:, -4:], test_data[1][:, -4:])
        
    return np.mean(bb_scores)

def makeTestData():
    testData = []
    
    for i in range(10):
        testData.append(parse_random_dataset(getData, getAnnotation, False))
        
    return testData

def testTiming(model, testData):
    total_time = 0.0
    total_acc = 0.0
    for i in range(10): 
        test_data = testData[i]

        testing_time = time.time()

        iou = test(model, test_data)

        time_taken = time.time()  - testing_time

        total_time += time_taken
        total_acc += iou

        print("\tFor this mini round, time: ", time_taken, " iou ", iou)

    avg_time = total_time / 10
    avg_acc = total_acc / 10
    
    print("Time on avg: ", avg_time, " IOU on avg: ", avg_acc)
    
    return avg_time, avg_acc

def trainModel():
    model = makeModel(INPUT_SHAPE, 8)
    
    train_start_time = time.process_time()

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=40,
        epochs=40,
        verbose=0)

    train_time = time.process_time() - train_start_time
    
    print("Training done!  " + str(train_time))
    
    return train_time, model

def getResults():
    testData = makeTestData()
    
    train_timing_results = np.empty(20)
    test_timing_results = np.empty(20)
    test_acc_results = np.empty(20)

    for i in range(20):
        print("Round ", i)

        train_time, model = trainModel()
        train_timing_results[i] = train_time
        
        test_time, test_acc = testTiming(model, testData)
        test_timing_results[i] = test_time
        test_acc_results[i] = test_acc

    print("\n\nTesting done!\n")

    print("Training time: ", np.mean(train_timing_results))
    print("Testing time: ", np.mean(test_timing_results))
    print("Testing acc: ", np.mean(test_acc_results))
    
    return train_timing_results, test_timing_results, test_acc_results

getResults()

Round  0
Training done!  35.296875
	For this mini round, time:  0.5949990749359131  iou  0.7982079
	For this mini round, time:  0.3279995918273926  iou  0.79770327
	For this mini round, time:  0.32900118827819824  iou  0.7957756
	For this mini round, time:  0.3319988250732422  iou  0.79259247
	For this mini round, time:  0.3340001106262207  iou  0.8008098
	For this mini round, time:  0.32899999618530273  iou  0.7947877
	For this mini round, time:  0.3260643482208252  iou  0.7876318
	For this mini round, time:  0.33300018310546875  iou  0.79497975
	For this mini round, time:  0.3320000171661377  iou  0.79766357
	For this mini round, time:  0.32700061798095703  iou  0.79631543
Time on avg:  0.3565063953399658  IOU on avg:  0.7956467270851135
Round  1
Training done!  31.5625
	For this mini round, time:  0.3840007781982422  iou  0.82142264
	For this mini round, time:  0.4849998950958252  iou  0.82043
	For this mini round, time:  0.31800007820129395  iou  0.8172153
	For this mini round, tim

Training done!  31.640625
	For this mini round, time:  0.3819706439971924  iou  0.8323379
	For this mini round, time:  0.3320286273956299  iou  0.82982874
	For this mini round, time:  0.33997011184692383  iou  0.8271714
	For this mini round, time:  0.3330252170562744  iou  0.8256016
	For this mini round, time:  0.33397459983825684  iou  0.8327709
	For this mini round, time:  0.3390014171600342  iou  0.8302362
	For this mini round, time:  0.3320271968841553  iou  0.82536286
	For this mini round, time:  0.42997288703918457  iou  0.8278449
	For this mini round, time:  0.33499860763549805  iou  0.82993734
	For this mini round, time:  0.3190011978149414  iou  0.8291213
Time on avg:  0.34759705066680907  IOU on avg:  0.8290213108062744
Round  12
Training done!  32.359375
	For this mini round, time:  0.381990909576416  iou  0.7942831
	For this mini round, time:  0.3230009078979492  iou  0.7955708
	For this mini round, time:  0.32700037956237793  iou  0.78693473
	For this mini round, time:  0.

(array([35.296875, 31.5625  , 32.375   , 32.      , 31.484375, 32.203125,
        32.046875, 32.0625  , 32.21875 , 32.078125, 32.046875, 31.640625,
        32.359375, 32.140625, 31.53125 , 32.140625, 32.328125, 31.5625  ,
        32.3125  , 32.171875]),
 array([0.3565064 , 0.35050011, 0.33299863, 0.34409738, 0.34220002,
        0.34729996, 0.36130235, 0.34850497, 0.34969757, 0.34919996,
        0.34359992, 0.34759705, 0.34089944, 0.34230685, 0.39559979,
        0.34449735, 0.33049994, 0.33330226, 0.33320003, 0.34140007]),
 array([0.79564673, 0.81942683, 0.78304152, 0.81032712, 0.80737042,
        0.81001498, 0.82349818, 0.78783683, 0.80110649, 0.80373079,
        0.82099667, 0.82902131, 0.79073673, 0.81984126, 0.78564985,
        0.82300112, 0.80298665, 0.7612779 , 0.81659399, 0.81132541]))

In [ ]:
def trainModel():
    model = makeModel(INPUT_SHAPE, 8)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=40,
        epochs=40,
        verbose=0)
    
    return history, model

history, model = trainModel()

In [6]:
w, h, points = getData(os.getcwd() + r'\..\Caltech101_real\caltech101_objects\101_ObjectCategories\airplanes\image_0002.jpg')
inputPoints = points.reshape([1, 128,128,3])
preds = model.predict(inputPoints)[0]


minX = preds[4] * w
minY = preds[5] * h
maxX = preds[6] * w
maxY = preds[7] * h

points = cv2.resize(points, (w,h)) * 256
fig = plt.figure(figsize=(20,10))
plt.imshow((points).astype(np.uint8))

ax = plt.gca()
ax.add_patch(makeBB(os.getcwd() + r'\..\Caltech101_real\caltech101_annotations\Annotations\airplanes\annotation_0002.mat'))
ax.add_patch(Rectangle((minX, minY), maxX - minX, maxY - minY, edgecolor='green', facecolor='none', label="predicted"))
ax.legend(loc="upper left", prop={'size': 15})

NameError: name 'model' is not defined

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

print(np.mean(history.history['val_loss']))
print(np.mean(history.history['val_accuracy']))